In [535]:
import pandas as pd, numpy as np, matplotlib, matplotlib.pyplot as plt
from sklearn import preprocessing
import numpy as np
from sklearn.neighbors import KNeighborsClassifier

allData = pd.read_csv('2019PredictionsAndActual.csv').dropna()
allData[['', 'PA', 'H', 'AB', 'RBI', 'G', '2B']] = preprocessing.scale(allData[['wRC', 'PA', 'H', 'AB', 'RBI', 'G', '2B']])


In [536]:
def getDataByLAP(fileName):
    positionData = pd.read_csv(fileName)

    position = positionData.loc[positionData['Season'] == 2018]
    position = position[['playerid', 'Team']].copy()
    position = pd.merge(position, allData, on='playerid').dropna()
    position[['runsPredicted', 'HRsPredicted', 'RBIsPredicted', 'AVGPredicted', 'opsPredicted']] = preprocessing.scale(position[['runsPredicted', 'HRsPredicted', 'RBIsPredicted', 'AVGPredicted', 'opsPredicted']])
    return position

In [537]:
specificData = {
    'al': [
        {'position': '1b', 'fileName': 'al-1b.csv', 'data': []},
        {'position': '2b', 'fileName': 'al-2b.csv', 'data': []},
        {'position': '3b', 'fileName': 'al-3b.csv', 'data': []},
        {'position': 'ss', 'fileName': 'al-ss.csv', 'data': []},
        {'position': 'of', 'fileName': 'al-of.csv', 'data': []},
        {'position': 'dh', 'fileName': 'al-dh.csv', 'data': []},
        {'position': 'c', 'fileName': 'al-c.csv', 'data': []},
    ],
    'nl': [
        {'position': '1b', 'fileName': 'nl-1b.csv', 'data': []},
        {'position': '2b', 'fileName': 'nl-2b.csv', 'data': []},
        {'position': '3b', 'fileName': 'nl-3b.csv', 'data': []},
        {'position': 'ss', 'fileName': 'nl-ss.csv', 'data': []},
        {'position': 'of', 'fileName': 'nl-of.csv', 'data': []},
        {'position': 'c', 'fileName': 'nl-c.csv', 'data': []},
    ]
}


In [538]:
# divide players by league and by position
for league in specificData:
    for position in specificData[league]:
        position['data'] = getDataByLAP(position['fileName'])

In [539]:
winners = {
    'al': {
        '1b': {
            'file': 'al-1b.csv',
            'stats': {
                '2016': { 'name': 'Miguel Cabrera', 'data': [] },
                '2017': { 'name': 'Eric Hosmer', 'data': [] },
                '2018': { 'name': 'Jose Abreu', 'data': [] }
            }
        },
        '2b': {
            'file': 'al-2b.csv',
            'stats': {
                '2016': { 'name': 'Jose Altuve', 'data': [] },
                '2017': { 'name': 'Jose Altuve', 'data': [] },
                '2018': { 'name': 'Jose Altuve', 'data': [] }
            }
        },
        '3b': {
            'file': 'al-3b.csv',
            'stats': {
                '2016': { 'name': 'Josh Donaldson', 'data': [] },
                '2017': { 'name': 'Jose Ramirez', 'data': [] },
                '2018': { 'name': 'Jose Ramirez', 'data': [] }
            }
        },
        'ss': {
            'file': 'al-ss.csv',
            'stats': {
                '2016': { 'name': 'Xander Bogaerts', 'data': [] },
                '2017': { 'name': 'Francisco Lindor', 'data': [] },
                '2018': { 'name': 'Francisco Lindor', 'data': [] }
            }
        },
        'ofs': {
            'file': 'al-of.csv',
            'stats': {
                '2016': {
                    'names': ['Mookie Betts', 'Mike Trout', 'Mark Trumbo'],
                    'data': []
                },
                '2017': {
                    'names': ['Aaron Judge', 'George Springer', 'Justin Upton'],
                    'data': []
                },
                '2018': {
                    'names': ['Mookie Betts', 'Mike Trout', 'J.D. Martinez'],
                    'data': []
                }
            }
        },
        'of1': {
            'file': 'al-of.csv',
            'stats': {
                '2016': { 'name': 'Mookie Betts', 'data': [] },
                '2017': { 'name': 'Aaron Judge', 'data': [] },
                '2018': { 'name': 'Mookie Betts', 'data': [] }
            }
        },
        'of2': {
            'file': 'al-of.csv',
            'stats': {
                '2016': { 'name': 'Mike Trout', 'data': [] },
                '2017': { 'name': 'George Springer', 'data': [] },
                '2018': { 'name': 'Mike Trout', 'data': [] }
            }
        },
        'of3': {
            'file': 'al-of.csv',
            'stats': {
                '2016': { 'name': 'Mark Trumbo', 'data': [] },
                '2017': { 'name': 'Justin Upton', 'data': [] },
                '2018': { 'name': 'J.D. Martinez', 'data': [] }
            }
        },
        'c': {
            'file': 'al-c.csv',
            'stats': {
                '2016': { 'name': 'Salvador Perez', 'data': [] },
                '2017': { 'name': 'Gary Sanchez', 'data': [] },
                '2018': { 'name': 'Salvador Perez', 'data': [] }
            }
        },
        'dh': {
            'file': 'al-dh.csv',
            'stats': {
                '2016': { 'name': 'David Ortiz', 'data': [] },
                '2017': { 'name': 'Nelson Cruz', 'data': [] },
                '2018': { 'name': 'J.D. Martinez', 'data': [] }    
            }
        }
    },
    'nl': {
        '1b': {
            'file': 'nl-1b.csv',
            'stats': {
                '2016': { 'name': 'Anthony Rizzo', 'data': [] },
                '2017': { 'name': 'Paul Goldschmidt', 'data': [] },
                '2018': { 'name': 'Paul Goldschmidt', 'data': [] }
            }
        },
        '2b': {
            'file': 'nl-2b.csv',
            'stats': {
                '2016': { 'name': 'Daniel Murphy', 'data': [] },
                '2017': { 'name': 'Daniel Murphy', 'data': [] },
                '2018': { 'name': 'Javier Baez', 'data': [] }
            }
        },
        '3b': {
            'file': 'nl-3b.csv',
            'stats': {
                '2016': { 'name': 'Nolan Arenado', 'data': [] },
                '2017': { 'name': 'Nolan Arenado', 'data': [] },
                '2018': { 'name': 'Nolan Arenado', 'data': [] }
            }
        },
        'ss': {
            'file': 'nl-ss.csv',
            'stats': {
                '2016': { 'name': 'Corey Seager', 'data': [] },
                '2017': { 'name': 'Corey Seager', 'data': [] },
                '2018': { 'name': 'Trevor Story', 'data': [] }
            }
        },
        'ofs': {
            'file': 'nl-of.csv',
            'stats': {
                '2016': {
                    'names': ['Christian Yelich', 'Yoenis Cespedes', 'Charlie Blackmon'],
                    'data': []
                },
                '2017': {
                    'names': ['Marcell Ozuna', 'Giancarlo Stanton', 'Charlie Blackmon'],
                    'data': []
                },
                '2018': {
                    'names': ['Christian Yelich', 'Nick Markakis', 'David Peralta'],
                    'data': []
                }
            }
        },
        'of1': {
            'file': 'nl-of.csv',
            'stats': {
                '2016': { 'name': 'Christian Yelich', 'data': [] },
                '2017': { 'name': 'Marcell Ozuna', 'data': [] },
                '2018': { 'name': 'Christian Yelich', 'data': [] }
            }
        },
        'of2': {
            'file': 'nl-of.csv',
            'stats': {
                '2016': { 'name': 'Yoenis Cespedes', 'data': [] },
                '2017': { 'name': 'Giancarlo Stanton', 'data': [] },
                '2018': { 'name': 'Nick Markakis', 'data': [] }
            }
        },
        'of3': {
            'file': 'nl-of.csv',
            'stats': {
                '2016': { 'name': 'Charlie Blackmon', 'data': [] },
                '2017': { 'name': 'Charlie Blackmon', 'data': [] },
                '2018': { 'name': 'David Peralta', 'data': [] }
            }
        },
        'c': {
            'file': 'nl-c.csv',
            'stats': {
                '2016': { 'name': 'Wilson Ramos', 'data': [] },
                '2017': { 'name': 'Buster Posey', 'data': [] },
                '2018': { 'name': 'J.T. Realmuto', 'data': [] }
            }
        }
    },
}

In [540]:
for league in winners:
    for position in winners[league]:
        if position == 'of':
            continue
        fileName = winners[league][position]['file']
                
        for year in winners[league][position]['stats']:
            data = pd.read_csv(fileName)
            data = data.loc[data['Season'] == int(year)]
#             data[['R', 'HR', 'RBI', 'AVG', 'OPS']] = preprocessing.scale(data[['R', 'HR', 'RBI', 'AVG', 'OPS']])
            
            if position != 'ofs':
                data['Award'] = np.where(data['Name'] == winners[league][position]['stats'][year]['name'], 1, 0)
                winners[league][position]['stats'][year]['playerdata'] = data[['Season', 'Name', 'playerid', 'R', 'HR', 'RBI', 'AVG', 'OPS', 'Award']]
                data = data.loc[data['Name'] == winners[league][position]['stats'][year]['name']]
                winner = data.loc[data['Name'] == winners[league][position]['stats'][year]['name']]
                winner = winner[['R', 'HR', 'RBI', 'AVG', 'OPS']].copy()
                winners[league][position]['stats'][year]['data'] = winner
            else:
                for index, row in data.iterrows():
                    if row['Name'] in winners[league][position]['stats'][year]['names']:
                        data.at[index, 'Award'] = 1
                    else:
                        data.at[index, 'Award'] = 0
                winners[league][position]['stats'][year]['playerdata'] = data[['Season', 'Name', 'playerid', 'R', 'HR', 'RBI', 'AVG', 'OPS', 'Award']]
                winner = data.loc[data['Award'] == 1]
                winner = winner[['R', 'HR', 'RBI', 'AVG', 'OPS']].copy()
                winners[league][position]['stats'][year]['data'] = winner
        winners[league][position]['average'] = pd.concat((winners[league][position]['stats']['2016']['data'], winners[league][position]['stats']['2017']['data']))
        winners[league][position]['average'] = pd.concat((winners[league][position]['average'], winners[league][position]['stats']['2018']['data']))
        winners[league][position]['average'] = winners[league][position]['average'].mean(axis=0)
        winners[league][position]['average'] = pd.DataFrame(data=winners[league][position]['average']).T
        frames = [winners[league][position]['stats']['2016']['playerdata'], winners[league][position]['stats']['2017']['playerdata'], winners[league][position]['stats']['2018']['playerdata']]
        winners[league][position]['playerdata'] = pd.concat(frames)
#     winners[league]['of'] = {}
#     of1 = pd.DataFrame(data=winners[league]['of1']['average'])
#     of2 = pd.DataFrame(data=winners[league]['of2']['average'])
#     of3 = pd.DataFrame(data=winners[league]['of3']['average'])
#     winners[league]['of']['average'] = pd.concat((of1, of2))
#     winners[league]['of']['average'] = pd.concat((winners['al']['of']['average'], of3))
#     winners[league]['of']['average'] = winners[league]['of']['average'].mean(axis=0)
#     winners[league]['of']['average'] = pd.DataFrame(data=winners[league]['of']['average']).T

# print(winners['al']['ofs']['stats']['2016']['playerdata'])
# bruh = bruh.loc(bruh['Award'] == 1)
# print(bruh)
winners['al']['of1']['stats']['2016']['data']

,R,HR,RBI,AVG,OPS
11,122,31,113,0.318,0.897


In [541]:
for league in winners:
    otherLeague = 'al'
    if league == 'al':
        otherLeague = 'nl'

    for position in winners[league]:
        frames = []
        temp = []
        if position == 'of' or position == 'of1' or position == 'of2' or position == 'of3':
            continue
        
        if position != 'dh':
            if position == 'ofs':
                frames.append(pd.DataFrame(data=winners[otherLeague]['of1']['stats'][year]['data']))
                frames.append(pd.DataFrame(data=winners[otherLeague]['of2']['stats'][year]['data']))
                frames.append(pd.DataFrame(data=winners[otherLeague]['of3']['stats'][year]['data']))
                temp = pd.concat(frames)
                temp['Award'] = 1
                train_x = pd.concat((winners[league][position]['playerdata'][['R', 'HR', 'RBI', 'AVG', 'OPS']],temp[['R', 'HR', 'RBI', 'AVG', 'OPS']]), axis=0)
                train_y = pd.concat((winners[league][position]['playerdata'][['Award']],temp[['Award']]))
            else:
                for year in winners[otherLeague][position]['stats']:
                    frames.append(pd.DataFrame(data=winners[otherLeague][position]['stats'][year]['data']))
                temp = pd.concat(frames)
                temp['Award'] = 1
                train_x = pd.concat((winners[league][position]['playerdata'][['R', 'HR', 'RBI', 'AVG', 'OPS']],temp[['R', 'HR', 'RBI', 'AVG', 'OPS']]), axis=0)
                train_y = pd.concat((winners[league][position]['playerdata'][['Award']],temp[['Award']]))
        else:
            train_x = winners[league][position]['playerdata'][['R', 'HR', 'RBI', 'AVG', 'OPS']]
            train_y = winners[league][position]['playerdata'][['Award']]
        for i, p in enumerate(specificData[league]):
            if specificData[league][i]['position'] == position:
                print(position)
                test = specificData[league][i]['data'][['playerid', 'Name', 'runsPredicted', 'HRsPredicted', 'RBIsPredicted', 'AVGPredicted', 'opsPredicted']]
                test = test.rename(columns={'runsPredicted': 'R', 'HRsPredicted': 'HR', 'RBIsPredicted': 'RBI', 'AVGPredicted': 'AVG', 'opsPredicted': 'OPS' })
                classifier = KNeighborsClassifier(n_neighbors=1)
                classifier.fit(train_x, train_y.values.ravel())

                for index, row in test.iterrows():
                    res = classifier.predict([[row['R'], row['HR'], row['RBI'], row['AVG'], row['OPS']]]) == [1]
                    if res == 1:
                        print(row['Name'])
            if specificData[league][i]['position'] == 'of' and position == 'ofs':
                print(position)
                test = specificData[league][i]['data'][['playerid', 'Name', 'runsPredicted', 'HRsPredicted', 'RBIsPredicted', 'AVGPredicted', 'opsPredicted']]
                test = test.rename(columns={'runsPredicted': 'R', 'HRsPredicted': 'HR', 'RBIsPredicted': 'RBI', 'AVGPredicted': 'AVG', 'opsPredicted': 'OPS' })
                classifier = KNeighborsClassifier(n_neighbors=1)
                classifier.fit(train_x, train_y.values.ravel())

                for index, row in test.iterrows():
                    res = classifier.predict([[row['R'], row['HR'], row['RBI'], row['AVG'], row['OPS']]]) == [1]
                    if res == 1:
                        print(row['Name'])

1b
2b
3b
ss
Manny Machado
ofs
c
dh
1b
Jesus Aguilar
Joey Votto
2b
DJ LeMahieu
3b
Nolan Arenado
ss
Trevor Story
ofs
Charlie Blackmon
c
Yadier Molina


In [469]:
winners['al']['of1']

{'average':           R        HR       RBI       AVG       OPS
 0  2.738011  2.309228  1.965268  1.890589  2.573141,
 'file': 'al-of.csv',
 'playerdata':      Season                Name  playerid         R        HR       RBI  \
 5      2016          Mike Trout     10155  2.632329  1.462479  1.900055   
 8      2016       J.D. Martinez      6184  0.474869  0.784509  0.703170   
 9      2016         Nelson Cruz      2434  1.553599  2.818420  2.087069   
 11     2016        Mookie Betts     13611  2.592376  1.656185  2.386290   
 15     2016        Tyler Naquin     13359 -0.204331  0.009685 -0.231897   
 19     2016         Chris Young      3882 -1.123249 -0.474579 -0.942547   
 20     2016     Robbie Grossman      5254 -0.324190 -0.280873 -0.456312   
 28     2016      Carlos Beltran       589  0.634681  1.462479  1.638237   
 29     2016         Mark Trumbo      6876  1.473693  3.205831  2.199277   
 33     2016        Jose Ramirez     13510  1.074164 -0.280873  1.002391   
 34     20